In [38]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [39]:
appRecord = pd.read_csv("application_record.csv");
creditRecord = pd.read_csv("credit_record.csv");

In [40]:
appRecord.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [41]:
creditRecord.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [42]:
data = pd.merge(
    appRecord,
    creditRecord,
    how="inner"
)

In [43]:
data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,C
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-1,C
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-2,C
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-3,C
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-4,C


In [44]:
data.info

<bound method DataFrame.info of              ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
1       5008804           M            Y               Y             0   
2       5008804           M            Y               Y             0   
3       5008804           M            Y               Y             0   
4       5008804           M            Y               Y             0   
...         ...         ...          ...             ...           ...   
777710  5150337           M            N               Y             0   
777711  5150337           M            N               Y             0   
777712  5150337           M            N               Y             0   
777713  5150337           M            N               Y             0   
777714  5150337           M            N               Y             0   

        AMT_INCOME_TOTAL NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0  

In [66]:
X = data.iloc[:data.size,18:19];
X.head()

,MONTHS_BALANCE
0,0
1,-1
2,-2
3,-3
4,-4


In [61]:
y = data.iloc[:data.size,19:20];
y.head()

,STATUS
0,C
1,C
2,C
3,C
4,C


In [62]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
label_encoder =LabelEncoder();
y['STATUS'] = label_encoder.fit_transform(y['STATUS']);

In [69]:
y.head()

,STATUS
0,6
1,6
2,6
3,6
4,6


In [75]:
y = np.array(y)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [76]:
# as a test, make a forest model with just a single feature. 
clf = RandomForestClassifier(max_depth=2, random_state=0);
clf.fit(X, y)

C:\Users\yun91\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_depth=2, random_state=0)

In [78]:
# see the first tree within the forest
print(clf.estimators_[0])

DecisionTreeClassifier(max_depth=2, max_features='auto', random_state=209652396)


In [80]:
# export as dot file
# on the command line, type dot tree.dot -Tpng -o tree.png to conver into png. 
from sklearn.tree import export_graphviz;
import os;
localFilePath = "C:/Users/yun91/Documents/GitHub/vis/data"
export_graphviz(
    clf.estimators_[0],
    out_file = os.path.join(localFilePath,"tree.dot"),
    rounded = True,
    filled = True)  